# Implementation Tasks

## Task 1

### 1.a

So the goal is to explore the bit fixed point bitwidth from 4 to 32, hence we would explore the accuracy against the memory footprint. This should also be done for QAT and PTQ models. So the graph should have two legends, and should be memory footprint against accuracy. There should also be two other graphs which are fixed point format against accuracy, and fixed point format against memory footprint.

In [7]:
from pathlib import Path
from chop import MaseGraph
import pandas as pd

tokenizer_checkpoint = "bert-base-uncased"
dataset_name = "imdb"

In [5]:
mg = MaseGraph.from_checkpoint(f"{Path.cwd()}/mase/tasks/tutorial2/tutorial_2_lora")

/rds/general/user/oa321/home/mase/src/chop/ir/graph/mase_graph.py:380: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model = torch.load(f)
WARNING  Node finfo not fou

In [2]:
from chop.tools import get_tokenized_dataset, get_trainer

dataset, tokenizer = get_tokenized_dataset(
    dataset=dataset_name,
    checkpoint=tokenizer_checkpoint,
    return_tokenizer=True
)

trainer = get_trainer(
    model=mg.model,
    tokenized_dataset=dataset,
    tokenizer=tokenizer,
    evaluate_metric="accuracy",
)

INFO     Tokenizing dataset imdb with AutoTokenizer for bert-base-uncased.
/rds/general/user/oa321/home/mase/src/chop/tools/huggingface.py:157: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [3]:
import chop.passes as passes

qc = {
    "by": "type",
    "default": {
        "config":
        {
            "name": None
        }
    },
    "linear":{
        "config":{
            "name":"integer",
            # Data
            "data_in_width": 8,
            "data_in_frac_width": 4,
            # Weight
            "weight_width": 8,
            "weight_frac_width": 4,
            # Bias
            "bias_width": 8,
            "bias_frac_width": 4,
        }
    }
}

mg, _ = passes.quantize_transform_pass(mg, pass_args=qc)

In [4]:
eval_results = trainer.evaluate()
print(eval_results['eval_accuracy'])

0.82452


In [18]:
def search_int_frac(qatOrPtq: bool) -> None:
    mg = MaseGraph.from_checkpoint(f"{Path.cwd()}/mase/tasks/tutorial2/tutorial_2_lora")
    frac_diff = [["" for i in range(0, 28)] for i in range(0, 28)]
    frac_diff_acc = [["" for i in range(0, 28)] for i in range(0, 28)]
    qc_template = qc

    for i in range(4, 5):
        qc_template["linear"]["config"]["bias_width"] = i
        qc_template["linear"]["config"]["data_in_width"] = i
        qc_template["linear"]["config"]["weight_width"] = i
        for j in range(4, 5):
            print(i, j)
            qc_template['by'] = "type"
            qc_template["linear"]["config"]["data_in_frac_width"] = j
            qc_template["linear"]["config"]["weight_frac_width"] = j
            qc_template["linear"]["config"]["bias_frac_width"] = j
            mg, _ = passes.quantize_transform_pass(mg, qc_template)
            trainer = get_trainer(
                model=mg.model,
                tokenized_dataset=dataset,
                tokenizer=tokenizer,
                evaluate_metric="accuracy",
            )

            if (qatOrPtq):
                trainer.train()

            eval_results = trainer.evaluate()
            frac_diff[i-4][j-4] = eval_results
            frac_diff_acc[i-4][j-4] = eval_results['eval_accuracy']

    data = []

    for i in range(4, 5):
        for j in range(4, 5):
            data.append({
                "width" : i,
                "frac_width": j,
                "accuracy": frac_diff_acc[i-4][j-4]
            })

    df = pd.DataFrame(data)
    df.to_csv(f"{Path.cwd()}/mase/tasks/tutorial3/out_{'qat' if qatOrPtq else 'pqt'}.csv", index=False)




In [19]:
search_int_frac(qatOrPtq=False)
search_int_frac(qatOrPtq=True)

/rds/general/user/oa321/home/mase/src/chop/ir/graph/mase_graph.py:380: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model = torch.load(f)
WARNING  Node finfo not fou

4 4


/rds/general/user/oa321/home/mase/src/chop/tools/huggingface.py:157: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


/rds/general/user/oa321/home/mase/src/chop/ir/graph/mase_graph.py:380: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model = torch.load(f)
WARNING  Node finfo not fou

4 4


/rds/general/user/oa321/home/mase/src/chop/tools/huggingface.py:157: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
500,0.615800
1000,0.542700
1500,0.501600
2000,0.480800
2500,0.480900
3000,0.474000
